In [1]:
## Import Library ##
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, Flatten
from keras.models import Model

In [2]:
folder_path = '../input/petfinder-pawpularity-score/'
test_file = '../input/petfinder-pawpularity-score/test.csv'

x_col = 'path'
y_col = 'Pawpularity'
test_df = pd.read_csv(test_file)
test_df['path'] = test_df['Id'].map(lambda x: str(folder_path+'/test/'+x)+'.jpg')

print(test_df.shape)
test_df.head()

In [3]:
## Test Data Generator ##
img_width, img_height = 512,512
batch_size = 32
no_of_classes = 1

test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df, x_col=x_col,
    target_size=(img_width, img_height),class_mode=None,
    shuffle=False) 

In [4]:
## Model ##
def create_model():
    input_tensor = Input(shape = (img_width, img_height,3))
    base_model = InceptionV3(weights = None, include_top = False, input_tensor = input_tensor)
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Flatten()(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(no_of_classes)(x)
    
    model = Model(input_tensor, final_output)
    
    return model

In [5]:
## Eval ##
model = create_model()
model.load_weights("../input/petfinder/inceptionv3_model.h5")
predictions = model.predict(train_generator)

In [6]:
## Submission File ##
result_df = pd.DataFrame()
result_df['Id'] = test_df['Id']
result_df['Pawpularity'] = predictions
result_df.to_csv('submission.csv', index=False)